<a href="https://colab.research.google.com/github/FalpClinicaTI/prueba_githubAction/blob/main/Tarea1_Analizando_datos_ventas_FernandoValdes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install pyspark
!pip install pyarrow

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")
gdrive='/content/gdrive/MyDrive/BigData/Clase1'

Mounted at /content/gdrive


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import min as pyspark_min, max as pyspark_max
from pyspark.sql.functions import count, avg, col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import desc
from pyspark.sql import functions as F

spark=SparkSession.builder.master("local[*]").appName("tarea1").getOrCreate()
segmentacion = gdrive+'/Tabla_segmentacion.csv'
pricing = gdrive+'/Tabla_pricing.csv'

df_segmentacion = spark.read.csv(segmentacion, inferSchema=True, header =True,sep=";")
df_pricing = spark.read.csv(pricing, inferSchema=True, header =True,sep=";")

In [7]:
df_segmentacion.createOrReplaceTempView("segmentacion")
df_pricing.createOrReplaceTempView("pricing")

In [12]:
# 1- Cuántos registros tienen ambos archivos
df_count_seg = spark.sql("select count(*) as count_seg from segmentacion")
df_count_pricing = spark.sql("select count(*) as count_pricing from pricing")

count_seg = df_count_seg.first()["count_seg"]
count_pricing = df_count_pricing.first()["count_pricing"]

print("La cantidad de registros del archivo Segmentación es: " + str(count_seg) +
      ", y la cantidad de registros del archivo Precios es de: " + str(count_pricing) +
      " la cantidad total de ambos archivos es: " + str(count_seg + count_pricing))

La cantidad de registros del archivo Segmentación es: 2805667, y la cantidad de registros del archivo Precios es de: 12581326 la cantidad total de ambos archivos es: 15386993


In [13]:
# 2 - Identificar las familias de productos, concluyendo rangos mínimos y máximos de márgenes .
df_pri_select = df_pricing.select("FAMILIA", "MARGEN")

resultados_familias = df_pri_select.groupBy("FAMILIA").agg(
    F.min("MARGEN").alias("min_margen"),
    F.max("MARGEN").alias("max_margen")
)

resultados_familias.show(truncate=False)

+-------+------------------+------------------+
|FAMILIA|min_margen        |max_margen        |
+-------+------------------+------------------+
|12     |0.1306476481437908|0.6862996158770807|
|1      |0.1306476481437908|0.6543916706056292|
|13     |0.1306476481437908|0.5705853658536586|
|6      |0.1306476481437908|0.9984176477444924|
|3      |0.1306476481437908|0.5800000000000001|
|5      |0.1306476481437908|0.9989324223337248|
|9      |0.1306476481437908|0.7539181286549708|
|4      |0.1306476481437908|0.8433924466193962|
|8      |0.1306476481437908|0.9423213066112972|
|7      |0.1306476481437908|0.6961444508475835|
|10     |0.1306476481437908|0.9348571428571428|
|11     |0.1306476481437908|0.6968634975081676|
|2      |0.1306476481437908|0.9129213483146068|
+-------+------------------+------------------+



In [18]:
# 3 - Realizar la consulta SQL para contabilizar clientes por segmento y calcular el promedio
tuplas_seg = df_segmentacion.groupBy("SEGMENTO").agg(F.count("RUT_CLI").alias("NUM_CLIENTES_SEG"))
tuplas_pri = df_pricing.groupBy("SEGMENTO").agg(F.count("FAMILIA").alias("NUM_CLIENTES_PRI"))


resultados_segmento = tuplas_seg.join(tuplas_pri, "SEGMENTO")
resultados_segmento = resultados_segmento.withColumn("PROMEDIO_CLIENTES", (F.col("NUM_CLIENTES_SEG") + F.col("NUM_CLIENTES_PRI")) / 2)

# Imprimir resultados
resultados_segmento.show(truncate=False)

+--------+----------------+----------------+-----------------+
|SEGMENTO|NUM_CLIENTES_SEG|NUM_CLIENTES_PRI|PROMEDIO_CLIENTES|
+--------+----------------+----------------+-----------------+
|45307   |1               |578             |289.5            |
|35912   |1               |1445            |723.0            |
|35982   |1               |2312            |1156.5           |
|124647  |1               |578             |289.5            |
|124967  |1               |578             |289.5            |
|121854  |1               |578             |289.5            |
|33868   |3               |1734            |868.5            |
|36538   |4               |4046            |2025.0           |
|124861  |1               |1156            |578.5            |
|32855   |1               |1734            |867.5            |
|124743  |1               |1156            |578.5            |
|37111   |2               |1156            |579.0            |
|35820   |1               |867             |434.0      

In [21]:
# 4 Realizar la consulta SQL para contar rubros distintos
consulta_sql = """SELECT COUNT(DISTINCT RUBRO) AS NUMERO_DE_RUBROS  FROM pricing"""

resultados = spark.sql(consulta_sql)
resultados.show()

+----------------+
|NUMERO_DE_RUBROS|
+----------------+
|              17|
+----------------+



In [23]:
# 5 - Qué cliente es el que mayor precio tiene?
cliente_mayor_precio = df_pricing.orderBy(F.desc("MARGEN")).first()

print("Cliente con el mayor precio:")
print(cliente_mayor_precio)

Cliente con el mayor precio:
Row(SEGMENTO=1, SUCURSAL='E523', RUBRO='C5 ', FAMILIA=5, SUBFAMILIA=11, PRODUCTO=None, JERARQUIA=2, MARGEN=0.9989324223337248, MARGEN_PRE_APRO=0.9989324223337248)


In [25]:
# En base a las sucursales , cual tiene el mayor y menor margen ? ( en promedio)
consulta_sql = """
    SELECT SUCURSAL, AVG(MARGEN) AS PROMEDIO_MARGEN
    FROM pricing
    GROUP BY SUCURSAL
    ORDER BY PROMEDIO_MARGEN DESC
    LIMIT 1
"""
resultados_mayor_margen = spark.sql(consulta_sql)
print("Sucursal con el mayor margen:")
resultados_mayor_margen.show()

consulta_sql = """
    SELECT SUCURSAL, AVG(MARGEN) AS PROMEDIO_MARGEN
    FROM pricing
    GROUP BY SUCURSAL
    ORDER BY PROMEDIO_MARGEN ASC
    LIMIT 1
"""
resultados_menor_margen = spark.sql(consulta_sql)

print("Sucursal con el menor margen:")
resultados_menor_margen.show()

Sucursal con el mayor margen:
+--------+------------------+
|SUCURSAL|   PROMEDIO_MARGEN|
+--------+------------------+
|    E550|0.3260823808111714|
+--------+------------------+

Sucursal con el menor margen:
+--------+------------------+
|SUCURSAL|   PROMEDIO_MARGEN|
+--------+------------------+
|    E576|0.2753196305799647|
+--------+------------------+

